Tutorial: Training models in Azure Databricks and deploying them on Azure ML

This notebook demostrates how to train models in Azure Databricks (or any Databricks implementation) and deploying those models on Azure ML.

Training and tracking experiments in Azure Databricks with Model Registries in Azure ML: This example shows how to do training and tracking of models in Azure Databricks. Tracking of experiments happens here in the MLflow instance running on Azure Databricks. However, model registries are kept on Azure ML to allow quick model's deployment from a centralized location and registry of models.

Mount the training data from azure blob store to dbfs : /dbfs/mnt/training-data/diabetes-training

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",                      
       "fs.azure.account.oauth2.client.id": "<client-id>",
       "fs.azure.account.oauth2.client.secret": "<client-secert>",
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<tenate-id>/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

dbutils.fs.mount(
source = "abfss://container@storage.dfs.core.windows.net/",
mount_point = "/mnt/training-data",
extra_configs = configs)

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-416402486082469> in <module> 
 6 "fs.azure.createRemoteFileSystemDuringInitialization": "true"}
 7 
 ----> 8 dbutils.fs.mount(
 9 source = "abfss://databricks-data@vmaluml5712272503.dfs.core.windows.net/" , 
 10 mount_point = "/mnt/training-data" , 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling o375.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/training-data; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/training-data
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:756)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:776)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/training-data
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:461)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$1(MetadataManager.scala:826)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:607)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:815)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:469)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:114)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:54)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:53)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:53)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$4.applyOrElse(DbfsServerBackend.scala:347)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$4.applyOrElse(DbfsServerBackend.scala:306)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:120)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:147)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:147)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:102)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLoggin

Run next cell to install latest version of library

In [0]:
%pip install azureml-mlflow
%pip install azure-ai-ml
%pip install mlflow

Python interpreter will be restarted.
Collecting azureml-mlflow
 Using cached azureml_mlflow-1.47.0-py3-none-any.whl (811 kB)
Requirement already satisfied: python-dateutil<3.0.0,>=2.7.3 in /databricks/python3/lib/python3.8/site-packages (from azureml-mlflow) (2.8.1)
Collecting azure-storage-blob<=12.13.0,>=12.5.0
 Using cached azure_storage_blob-12.13.0-py3-none-any.whl (377 kB)
Collecting azure-mgmt-core<2.0.0,>=1.2.0
 Using cached azure_mgmt_core-1.3.2-py3-none-any.whl (26 kB)
Collecting azure-common<2.0.0,>=1.1
 Using cached azure_common-1.1.28-py2.py3-none-any.whl (14 kB)
Collecting azure-identity
 Using cached azure_identity-1.12.0-py3-none-any.whl (135 kB)
Requirement already satisfied: mlflow-skinny in /databricks/python3/lib/python3.8/site-packages (from azureml-mlflow) (1.24.0)
Collecting azure-core!=1.22.0,<2.0.0,>=1.8.0
 Using cached azure_core-1.26.1-py3-none-any.whl (172 kB)
Collecting msrest>=0.6.18
 Using cached msrest-0.7.1-py3-none-any.whl (85 kB)
Collecting jsonpickle
 Using cached jsonpickle-2.2.0-py2.py3-none-any.whl (39 kB)
Requirement already satisfied: cryptography in /databricks/python3/lib/python3.8/site-packages (from azureml-mlflow) (3.4.7)
Requirement already satisfied: requests>=2.18.4 in /databricks/python3/lib/python3.8/site-packages (from azure-core!=1.22.0,<2.0.0,>=1.8.0->azureml-mlflow) (2.25.1)
Collecting typing-extensions>=4.0.1
 Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
Requirement already satisfied: six>=1.11.0 in /databricks/python3/lib/python3.8/site-packages (from azure-core!=1.22.0,<2.0.0,>=1.8.0->azureml-mlflow) (1.15.0)
Requirement already satisfied: cffi>=1.12 in /databricks/python3/lib/python3.8/site-packages (from cryptography->azureml-mlflow) (1.14.5)
Requirement already satisfied: pycparser in /databricks/python3/lib/python3.8/site-packages (from cffi>=1.12->cryptography->azureml-mlflow) (2.20)
Requirement already satisfied: isodate>=0.6.0 in /databricks/python3/lib/python3.8/site-packages (from msrest>=0.6.18->azureml-mlflow) (0.6.0)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from msrest>=0.6.18->azureml-mlflow) (2020.12.5)
Requirement already satisfied: requests-oauthlib>=0.5.0 in /databricks/python3/lib/python3.8/site-packages (from msrest>=0.6.18->azureml-mlflow) (1.3.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core!=1.22.0,<2.0.0,>=1.8.0->azureml-mlflow) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core!=1.22.0,<2.0.0,>=1.8.0->azureml-mlflow) (2.10)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.18.4->azure-core!=1.22.0,<2.0.0,>=1.8.0->azureml-mlflow) (4.0.0)
Requirement already satisfied: oauthlib>=3.0.0 in /databricks/python3/lib/python3.8/site-packages (from requests-oauthlib>=0.5.0->msrest>=0.6.18->azureml-mlflow) (3.1.0)
Collecting msal<2.0.0,>=1.12.0
 Using cached msal-1.20.0-py2.py3-none-any.whl (90 kB)
Collecting msal-extensions<2.0.0,>=0.3.0
 Using cached msal_extensions-1.0.0-py2.py3-none-any.whl (19 kB)
Collecting PyJWT[crypto]<3,>=1.0.0
 Using cached PyJWT-2.6.0-py3-none-any.whl (20 kB)
Collecting portalocker<3,>=1.0
 Using cached portalocker-2.6.0-py2.py3-none-any.whl (15 kB)
Requirement already satisfied: click>=7.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow-skinny->azureml-mlflow) (7.1.2)
Requirement already satisfied: databricks-cli>=0.8.7 in /databricks/python3/lib/python3.8/site-packages (from mlflow-skinny->azureml-mlflow) (0.16.3)
Requirement already satisfied: cloudpickle in /databricks/python3/lib/python3.8/site-packages (from mlflow-skinny->azureml-mlflow) (1.6.0)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.8/site-packages (from mlflow-skinny->azureml-mlflow) (21.3)
Requi

Configure the following variables

In [0]:
aml_region = ""
subscription_id = ""
aml_resource_group = ""
aml_workspace_name = ""
adb_user_id = ""

In [0]:
azureml_mlflow_uri=f"azureml://{aml_region}.api.azureml.ms/mlflow/v1.0/subscriptions/{subscription_id}/resourceGroups/{aml_resource_group}/providers/Microsoft.MachineLearningServices/workspaces/{aml_workspace_name}"
print(azureml_mlflow_uri)

azureml://eastus.api.azureml.ms/mlflow/v1.0/subscriptions/a63019f4-2005-489a-92e3-5fce495bcdfb/resourceGroups/dataintelligencerg-eastUS/providers/Microsoft.MachineLearningServices/workspaces/Databricks_AML

In some cases we may want to keep doing tracking of experiments in the MLflow instance that comes with Azure Databricks. This is the case for instance of customers that were already using MLflow in Azure Databricks so they want to keep they existing experiments there. However, they may want to take adavantage of the deployment capabilities of Azure ML including managed inference solutions, no-code deployments, etc.

In [0]:
import warnings

warnings.simplefilter("ignore")

Configuring models' registry

MLflow allows us to segregate the instance where experiments are being tracked from the instance where models' are being tracked (or registered). The first tutorial is referred to Tracking URI while the second one is referred as Registry URI. By default, both of them are set to the same value, and in Azure Databricks, both of them are set to "databricks" meaning that tracking and model registries will happen inside of the MLflow instance that Databricks runs for us.

We are going to track the experiments in Azure Databricks, but model registries will be held in Azure ML. This will allow us to manage the model's lifecycle - including deployments - in Azure ML.

In [0]:
import mlflow
mlflow.set_registry_uri(azureml_mlflow_uri)

Configuring the experiment

Tracking of experiments will happen in Azure Datbricks and hence we need to use the naming we use here.

In [0]:
mlflow.set_experiment(experiment_name=f"/Users/{adb_user_id}/diabetes-prediction-databricks")

InteractiveBrowserCredential.get_token failed: Failed to open a browser
2022/11/28 09:30:30 INFO mlflow.tracking.fluent: Experiment with name '/Users/vinitamalu@microsoft.com/diabetes-prediction-databricks' does not exist. Creating a new experiment.
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
Out[9]: <Experiment: artifact_location='', creation_time=1669627830552, experiment_id='caf30461-4746-480b-8482-e466e6865a0d', last_update_time=None, lifecycle_stage='active', name='/Users/vinitamalu@microsoft.com/diabetes-prediction-databricks', tags={}>

Exploring the data

In [0]:
import pandas as pd
df_diabetes = pd.read_csv('/dbfs/mnt/training/*.csv')
df_diabetes.head()

Out[10]:

,PatientID,Pregnancies,PlasmaGlucose,DiastolicBloodPressure,TricepsThickness,SerumInsulin,BMI,DiabetesPedigree,Age,Diabetic
0,1354778,0,171,80,34,23,43.509726,1.213191,21,0
1,1147438,8,92,93,47,36,21.240576,0.158365,23,0
2,1640031,7,115,47,52,35,41.511523,0.079019,23,0
3,1883350,9,103,78,25,304,29.582192,1.282870,43,1
4,1424119,1,85,59,27,35,42.604536,0.549542,22,0


Training a diabetes prediction regression model

In [0]:
from sklearn.model_selection import train_test_split
# Separate features and labels
X, y = df_diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, df_diabetes['Diabetic'].values

# Split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)


We are going to use autologging capabilities in MLflow to track parameters and metrics.

In [0]:
mlflow.autolog() #enable logging for sklearn models

2022/11/28 09:31:14 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/11/28 09:31:15 INFO mlflow._spark_autologging: Autologging successfully enabled for spark.
2022/11/28 09:31:15 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2022/11/28 09:31:15 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.ml.

Create a model & train it

In [0]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import roc_auc_score

# Set regularization hyperparameter
reg = 0.01

with mlflow.start_run() as run:
    model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)
    # calculate accuracy
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)
    # calculate AUC
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print("Accuracy: %.2f%%" % (acc * 100.0))
    print("AUC: %.2f%%" % (auc * 100.0))

InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
Accuracy: 77.40%
AUC: 84.84%
InteractiveBrowserCredential.get_token failed: Failed to open a browser

Registering the model in Azure ML

So far, our model is trained and tracked inside of the MLflow instance in Azure Databricks. Now we want to register this model in Azure ML to manage the life cicle there. However, if we try to register the model as we usually do using the sintax mlflow.register_model(model_uri=f"runs:/{run.info.run_id}/model"). you will found an error. The reason why this is happening is related to where runs are being stored.

Right now runs are being stored in Azure Databricks and models in Azure ML. If you try to create a registered model from a Run, Azure ML don't have any way to guess how to get access to the runs, that are stored in a different service. because of that, you can't use runs:/ URI for registering models.

To overcome this limitation, we have to register the model from the artifacts themselfs, which we can achieve by first downloading them.

In [0]:
client = mlflow.tracking.MlflowClient()
model_path = client.download_artifacts(run.info.run_id, path="model")

model_path is a local path to the artifacts representing the MLmodel created. We can use this artifacts to register the model now:

In [0]:
mlflow.register_model(
    model_uri=f"file://{model_path}", name="databricks-diabetes-prediction"
)

InteractiveBrowserCredential.get_token failed: Failed to open a browser
Registered model 'databricks-diabetes-prediction' already exists. Creating a new version of this model...
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
InteractiveBrowserCredential.get_token failed: Failed to open a browser
2022/11/28 09:37:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: databricks-diabetes-prediction, version 1
Created version '1' of model 'databricks-diabetes-prediction'.
Out[16]: <ModelVersion: creation_timestamp=1669628232008, current_stage='None', description='', last_updated_timestamp=1669628232008, name='databricks-diabetes-prediction', run_id='', run_link='', source='azureml://artifacts/LocalUpload/221128T093710-01597558/model', status='READY', status_message='', tags={}, user_id='', version='1'>

Summary:

In this tutorial we leaned how we can us mlflow for AML* ADB integration. We train and track a model in databricks & use Azure ML model registery to register the model .
You'll find your model in Azure ML workspace. You can check Job & model UI configuration for more details.